In [ ]:
import sys
sys.path.append('/g/g92/miao1/Development/llsmvis')
sys.path.append('/Users/yi10/Desktop/Research/Software/T-llsmvis/')
sys.path.append('/g/g92/yi10')
import llsmvis
from llsmvis.extensions import hp3d
from llsmvis.extensions.hp3d import surface_morphometrics
import os
from skimage import io
import numpy as np
import matplotlib.pyplot as plt
import copy

In [ ]:
# ticket information (the output files from the interactive trimming results, see E1 for details)
tpath='/p/lscratchh/yi10/hp3d/CroppingData'
thead='2021-10-29-cell3'  
tlist=hp3d.croproi3d.get_trimmed_tiff_list(tpath, thead)


In [ ]:
# h5fname='/p/lscratchh/yi10/hp3d/2021-10-29-cell3/hp3ddata_2021-10-29-cell3.hdf5'

hp3ddata_path='/p/lscratchh/yi10/hp3d/'+thead
d=hp3d.hp3ddata.HP3Ddata(fpath=hp3ddata_path,dfnamehead='hp3ddata_'+thead,initialize=False,verbose=False)


In [ ]:
lginds=[np.nan]*50
lgs=[0]*50


In [ ]:
lgs[0]=1458
lgs[10]=1270
lgs[20]=1070
lgs[30]=860
lgs[35]=786
lgs[45]=700
lgs[49]=658
lgs[1:10]=np.linspace(lgs[0],lgs[10],9)
lgs[11:20]=np.linspace(lgs[10],lgs[20],9)
lgs[21:30]=np.linspace(lgs[20],lgs[30],9)
lgs[31:35]=np.linspace(lgs[30],lgs[35],4)
lgs[36:45]=np.linspace(lgs[35],lgs[45],9)
lgs[46:49]=np.linspace(lgs[45],lgs[49],3)


In [ ]:
for tind in np.arange(50):
    hs=np.asarray(d.h5f['[G02] voxel value histogram counts/T'+str(tind)])
    lbind=np.int(d.h5f['[D7] threshold lower bound index'][tind])
    spind=np.int(d.h5f['[D3] threshold saddle point index'][tind])
    ubind=np.int(d.h5f['[D5] threshold upper bound index'][tind])
    bc=np.asarray(d.h5f['[G01] voxel value bin centers/T'+str(tind)])
    plt.plot(bc,(hs+1)**0.01+tind*0.01)
    lgind=np.where(bc>=lgs[tind])[0][0]
    plt.plot(bc[lbind], (hs[lbind]+1)**0.01+tind*0.01,'ro')
    plt.plot(bc[spind], (hs[spind]+1)**0.01+tind*0.01,'bo')
    plt.plot(bc[ubind], (hs[ubind]+1)**0.01+tind*0.01,'go')
    plt.plot(bc[lgind], (hs[lgind]+1)**0.01+tind*0.01,'mo')

In [ ]:
# plot thresholds and profiles
for tind in np.arange(50):
    lb=d.h5f['[D6] threshold lower bound'][tind]
    sp=d.h5f['[D2] threshold saddle point'][tind]
    ub=d.h5f['[D4] threshold upper bound'][tind]

    lbind=np.int(d.h5f['[D7] threshold lower bound index'][tind])
    spind=np.int(d.h5f['[D3] threshold saddle point index'][tind])
    ubind=np.int(d.h5f['[D5] threshold upper bound index'][tind])
    bc=np.asarray(d.h5f['[G01] voxel value bin centers/T'+str(tind)])
    lgind=np.where(bc>=lgs[tind])[0][0]
    lg=bc[lgind]

    hs=np.asarray(d.h5f['[G02] voxel value histogram counts/T'+str(tind)])
    fpath=tlist[tind]
    output_dir='/p/lscratchh/yi10/test'
    k0=io.imread(fpath)
    k=copy.deepcopy(k0)
    k[np.where(k<lg)]=0
    
    # print(bc[lgind])
    plt.figure()
    plt.imshow(np.max(k,axis=0))


In [ ]:
surface_smoothing_steps=10
output_file = hp3d.surface_morphometrics.extract_surface(lg, fpath=fpath, 
                                                         output_dir=output_dir, 
                                                         kernel_size=3, 
                                                         save_mask=True, 
                                                         surface_smoothing_steps=surface_smoothing_steps,
                                                        largest_connected_component=False,
                                                        output_fname='Legionella.stl')